<a href="https://colab.research.google.com/github/sbbwoy/ESAA/blob/main/220328_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1-a. MNIST 데이터를 훈련, 검증, 테스트 데이터로 나눈다.

In [14]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784',version=1)

In [16]:
from sklearn.model_selection import train_test_split

X, y =mnist['data'], mnist['target']
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

1-b. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시킨다. (n_estimators=100, random_state=42)

In [20]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier  # 랜덤 포레스트 분류기, 엑스트라 트리 분류기
from sklearn.svm import LinearSVC  # SVM 분류기
from sklearn.neural_network import MLPClassifier  # MLP 분류기(Multi-layer Perceptron Classifier, 다층 인공 신경망 분류기)

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators: 생성할 의사결정 나무 개수 
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)

svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [29]:
from sklearn.metrics import accuracy_score

In [30]:
estimators = [rf_clf, et_clf, svm_clf, mlp_clf]

for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)
    print('accuracy_score: ', accuracy_score(y_test, y_pred))

Training the RandomForestClassifier(random_state=42)
accuracy_score:  0.9705
Training the ExtraTreesClassifier(random_state=42)
accuracy_score:  0.9722
Training the LinearSVC(max_iter=100, random_state=42, tol=20)
accuracy_score:  0.865
Training the MLPClassifier(random_state=42)
accuracy_score:  0.9675


1-c. 이들을 직접 투표 분류기를 사용하는 앙상블로 연결한다.

In [24]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('etr', et_clf), ('svc', svm_clf), ('mlp', mlp_clf)], voting='hard')  
# 직접 투표 분류기 (간접 투표 분류기는 'soft')

In [32]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('etr', ExtraTreesClassifier(random_state=42)),
                             ('svc',
                              LinearSVC(max_iter=100, random_state=42, tol=20)),
                             ('mlp', MLPClassifier(random_state=42))])

In [33]:
from sklearn.metrics import accuracy_score

y_pred = voting_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9711

1-d. 다른 분류기보다 성능이 많이 떨어졌던 선형 SVM 제거 후 VotingClassifier를 다시 평가

In [34]:
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('etr', et_clf), ('mlp', mlp_clf)], voting='hard') 

In [36]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('etr', ExtraTreesClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [37]:
y_pred = voting_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9741

1-e. 간접 투표 분류기를 사용하여 앙상블로 연결.

In [39]:
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('etr', et_clf), ('mlp', mlp_clf)], voting='soft') 

In [40]:
voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9722